In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#Ovde se uvoze biblioteke koje će se koristiti u kodu. 
#pandas za rad sa podacima, numpy za numeričke operacije, 
#torch za rad s PyTorch tenzorima, 
#BertForQuestionAnswering za BERT model koji odgovara na pitanja, i BertTokenizer za tokenizaciju.

In [2]:
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
pd.set_option('display.max_colwidth', None)
coqa.head()

#Učitavaju se podaci iz CoQA skupa podataka koji su u JSON formatu. 
#coqa.head() se koristi da prikaže prvih nekoliko redova kako biste videli kako izgledaju podaci

version  \
0        1   
1        1   
2        1   
3        1   
4        1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [3]:
del coqa["version"]

#Izbacuje se kolona "version" iz DataFrame-a, 

In [4]:
cols = ["text","question","answer"]
#definišu se potrebne kolone za novi DataFrame (new_df): "text", "question", "answer".


comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)
        
        
new_df = pd.DataFrame(comp_list, columns=cols) 
#Pravi se novi DataFrame (new_df) koji sadrži tekst priče, pitanje i odgovor. 
#Podaci se čuvaju u CSV fajlu pod nazivom "CoQA_data.csv".

new_df.to_csv("CoQA_data.csv", index=False)
#cuvanje dataframe-a u csv fajl za buduce ucitavanje

In [5]:
data = pd.read_csv("CoQA_data.csv")
#Ucitavanje podataka iz lokalnog CSV fajla
data.head()

,text,question,answer
0,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items. \n\nScholars have traditionally divided the history of the library into five periods, Pre-Lateran, Lateran, Avignon, Pre-Vatican and Vatican. \n\nThe Pre-Lateran period, comprising the initial days of the library, dated from the earliest days of the Church. Only a handful of volumes survive from this period, though some are very significant.",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items. \n\nScholars have traditionally divided the history of the library into five periods, Pre-Lateran, Lateran, Avignon, Pre-Vatican and Vatican. \n\nThe Pre-Lateran period, comprising the initial days of the library, dated from the earliest days of the Church. Only a handful of volumes survive from this period, though some are very significant.",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items. \n\nScholars have t

In [6]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Učitavaju se prethodno obučeni BERT model i odgovarajući tokenizator.

In [7]:
def question_answer(question, text):
    # Tokenizacija pitanja i teksta kao para
    input_ids = tokenizer.encode(question, text)

    # Pretvaranje tokena u stringove
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Pronalaženje prvog pojavljivanja tokena [SEP]
    # Podaci se šalju Bert-u u sledećem obliku:[CLS] <pitanje> [SEP] <tekst>
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    # Broj tokena u segmentu A (pitanje)
    num_seg_a = sep_idx + 1

    # Broj tokena u segmentu B (tekst)
    num_seg_b = len(input_ids) - num_seg_a

    # Lista nula i jedinica za segmentne ugnježdenosti
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    assert len(segment_ids) == len(input_ids)

    # Prolazak kroz model koristeći input_ids i segment_ids
    #torch.tensor([input_ids]): Pravi se PyTorch tenzor od list input_ids. BERT model očekuje da mu se podaci proslede u obliku 
    #tenzora.
    #token_type_ids=torch.tensor([segment_ids]): Tenzor segment_ids se takođe konvertuje u PyTorch tenzor i prosleđuje modelu 
    #kao argument 
    output = model( torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )

    # Rekonstrukcija odgovora
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    

    if answer_end >= answer_start:
        answer = tokens[answer_start]

        # Spajanje tokena koji počinju sa "##"
        for i in range(answer_start + 1, answer_end + 1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
    else: 
        answer = tokens[answer_start]

    # Ako odgovor počinje sa "[CLS]", znači da odgovor nije pronađen
    if answer.startswith("[CLS]" or " "):
        answer = "Unable to find the answer to your question."

    # Prikaz predviđenog odgovora
    print("\nPredicted answer:\n{}".format(answer.capitalize()))


In [8]:
text = """New York (CNN) -- More than 80 Michael Jackson collectibles -- including the late pop star's 
famous rhinestone-studded glove from a 1983 performance -- were auctioned off Saturday, reaping a total $2 million. 
Profits from the auction at the Hard Rock Cafe in New York's Times Square crushed pre-sale expectations of only 
$120,000 in sales. The highly prized memorabilia, which included items spanning the many stages of Jackson's career, 
came from more than 30 fans, associates and family members, who contacted Julien's Auctions to sell their gifts and 
mementos of the singer. Jackson's flashy glove was the big-ticket item of the night, fetching $420,000 from a buyer 
in Hong Kong, China. Jackson wore the glove at a 1983 performance during \"Motown 25,\" an NBC special where he 
debuted his revolutionary moonwalk. Fellow Motown star Walter \"Clyde\" Orange of the Commodores, who also performed 
in the special 26 years ago, said he asked for Jackson's autograph at the time, but Jackson gave him the glove instead.
"The legacy that [Jackson] left behind is bigger than life for me,\" Orange said. \"I hope that through that glove 
people can see what he was trying to say in his music and what he said in his music.\" Orange said he plans to give a 
portion of the proceeds to charity. Hoffman Ma, who bought the glove on behalf of Ponte 16 Resort in Macau, paid a 25 
percent buyer's premium, which was tacked onto all final sales over $50,000. Winners of items less than $50,000 paid a 
20 percent premium."""

question = "Where was the Auction held?"
question_answer(question, text)
#originalni odgovor iz skupa podataka
print("Original answer:\n", data.loc[data["question"] == question]["answer"].values[0])


Predicted answer:
Hard rock cafe in new york ' s times square
Original answer:
 Hard Rock Cafe


In [9]:
question = "What is the capital of France?"
question_answer(question, text)


Predicted answer:
Unable to find the answer to your question.


In [10]:

for index, row in data.head().iterrows():
    text = row['text']
    question = row['question']
    
    # Prikazivanje teksta, pitanja i originalnog odgovora
    print("\nTekst:\n{}".format(text))
    print("\nPitanje:\n{}".format(question))

    # Pitanje i odgovaranje koristeći funkciju question_answer
    question_answer(question, text)

    # Originalni odgovor iz podataka
    print("Originalni odgovor:\n{}".format(row['answer']))
    print("-" * 50)



Tekst:
The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. 

The Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. 

In March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. 

The Vatican Secret Archives were separated from the library at the beginning of 

In [13]:
text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")
while True:
    question_answer(question, text)
    
    flag = True
    flag_N = False
    
    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True
            
    if flag_N == True:
        break

Please enter your text: 
Arrays These are one of the most basic data structures, and they consist of a collection of items that are stored in a contiguous block of memory. Arrays are used to store data of the same type, and they are accessed using an index. Arrays are a very efficient data structure for storing and accessing data, and they are commonly used in many algorithms.   Linked List Linked lists are another common data structure, and they consist of a series of nodes that are connected together by links. Each node contains a value and a pointer to the next node in the list. Linked lists are often used when the size of the data is not known in advance or when the data needs to be inserted or deleted in the middle of the list.  Stacks and Queues Stacks and queues are data structures that are based on the concept of Last In, First Out (LIFO) and First In, First Out (FIFO), respectively. In a stack, items are added to and removed from the top of the stack, while in a queue, items a

In [ ]:
import tkinter as tk
from tkinter import messagebox


def question_answer1(question, text):
    # Tokenizacija pitanja i teksta kao para
    input_ids = tokenizer.encode(question, text)

    # Pretvaranje tokena u stringove
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Pronalaženje prvog pojavljivanja tokena [SEP]
    # Podaci se šalju Bert-u u sledećem obliku:[CLS] <pitanje> [SEP] <tekst>
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    # Broj tokena u segmentu A (pitanje)
    num_seg_a = sep_idx + 1

    # Broj tokena u segmentu B (tekst)
    num_seg_b = len(input_ids) - num_seg_a

    # Lista nula i jedinica za segmentne ugnježdenosti
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    assert len(segment_ids) == len(input_ids)

    # Prolazak kroz model koristeći input_ids i segment_ids
    #torch.tensor([input_ids]): Pravi se PyTorch tenzor od list input_ids. BERT model očekuje da mu se podaci proslede u obliku 
    #tenzora.
    #token_type_ids=torch.tensor([segment_ids]): Tenzor segment_ids se takođe konvertuje u PyTorch tenzor i prosleđuje modelu 
    #kao argument 
    output = model( torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )

    # Rekonstrukcija odgovora
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    

    if answer_end >= answer_start:
        answer = tokens[answer_start]

        # Spajanje tokena koji počinju sa "##"
        for i in range(answer_start + 1, answer_end + 1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
    else: 
        answer = tokens[answer_start]

    # Ako odgovor počinje sa "[CLS]", znači da odgovor nije pronađen
    if answer.startswith("[CLS]" or " "):
        answer = "Unable to find the answer to your question."

    # Prikaz predviđenog odgovora
    #print("\nPredicted answer:\n{}".format(answer.capitalize()))
    return answer.capitalize()


def question_answer_gui():
    tekst = unos_teksta.get("1.0", tk.END)  # Dobijanje celog teksta iz Text widget-a
    pitanje = unos_pitanja.get()
    odgovor = question_answer1(pitanje, tekst)
    prikaz_odgovora.delete("1.0", tk.END)  # Brisnje prethodnog odgovora
    prikaz_odgovora.insert(tk.END, odgovor.capitalize())  # Prikazivanje novog odgovora

def postavi_dodatno_pitanje():
    odgovor = messagebox.askyesno("Dodatno Pitanje", "Da li želite postaviti još jedno pitanje na osnovu ovog teksta?")
    if odgovor:
        unos_pitanja.delete(0, tk.END)
        prikaz_odgovora.delete("1.0", tk.END)  # Brisnje prethodnog odgovora

    else:
        messagebox.showinfo("Doviđenja!", "Doviđenja!")
        prozor.destroy()

# Glavni prozor
prozor = tk.Tk()
prozor.title("QA Model GUI")

# Postavljanje prozora kao full screen
prozor.attributes("-fullscreen", True)

# Unos teksta (koristi Text widget)
labela_teksta = tk.Label(prozor, text="Unesite tekst:")
labela_teksta.pack()

unos_teksta = tk.Text(prozor, height=30, width=100)  # Povećana visina Text widget-a
unos_teksta.pack()

# Unos pitanja
labela_pitanja = tk.Label(prozor, text="Unesite pitanje:")
labela_pitanja.pack()
unos_pitanja = tk.Entry(prozor, width=50)
unos_pitanja.pack()

# Dugmad
dugme_qa = tk.Button(prozor, text="Dobij odgovor", command=question_answer_gui)
dugme_qa.pack()

dugme_dodatnog_pitanja = tk.Button(prozor, text="Postavi dodatno pitanje", command=postavi_dodatno_pitanje)
dugme_dodatnog_pitanja.pack()

# Prikazivanje odgovora
labela_odgovora = tk.Label(prozor, text="Odgovor:")
labela_odgovora.pack()

prikaz_odgovora = tk.Text(prozor, height=5, width=50)  # Povećana visina Text widget-a
prikaz_odgovora.pack()

prozor.mainloop()
